In [23]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import image_dataset_from_directory
from keras.preprocessing import image
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten

In [24]:
# Loading training and validation data 
train_data = image_dataset_from_directory(
    directory="../data/raw/train/images",
    label_mode = 'categorical',
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=32
)

test_data = image_dataset_from_directory(
    directory="../data/raw/validation/images",
    label_mode = 'categorical',
    color_mode='rgb',
    image_size=(224, 224),
    batch_size=32
)

Found 1440 files belonging to 6 classes.
Found 360 files belonging to 6 classes.


In [25]:
# NORMALIZING THE IMAGES

def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    return image, label

train_input = train_data.map(preprocess)
test_input = test_data.map(preprocess)

In [26]:
AUTOTUNE = tf.data.AUTOTUNE

train_input = train_input.cache().prefetch(buffer_size=AUTOTUNE)
test_input = test_input.cache().prefetch(buffer_size=AUTOTUNE)

In [27]:
# Data Augmentation
augment_step = Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomFlip("vertical"),
    tf.keras.layers.RandomRotation(0.05),
    tf.keras.layers.RandomZoom(0.1)
])

In [28]:
base_model = MobileNetV2(weights='imagenet', include_top='false', input_shape=(224, 224,3))

In [29]:
base_model.trainable = False

In [31]:
model = Sequential()

model.add(keras.layers.Input(shape=(224, 224,3)))

model.add(augment_step)

model.add(base_model)

model.add(Dense(6, activation='softmax'))

In [32]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_4 (Sequential)       │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1000)           │     3,538,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │         6,006 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,544,990 (13.52 MB)

 Trainable params: 6,006 (23.46 KB)

 Non-trainable params: 3,538,984 (13.50 MB)

In [33]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [35]:
history = model.fit(train_input, epochs=20, validation_data=test_input)

Epoch 1/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 29s 642ms/step - accuracy: 0.8056 - loss: 1.6450 - val_accuracy: 0.7056 - val_loss: 1.6381
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 30s 674ms/step - accuracy: 0.8285 - loss: 1.6315 - val_accuracy: 0.7111 - val_loss: 1.6260
Epoch 3/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 683ms/step - accuracy: 0.8139 - loss: 1.6159 - val_accuracy: 0.7111 - val_loss: 1.6132
Epoch 4/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 698ms/step - accuracy: 0.8333 - loss: 1.6050 - val_accuracy: 0.7083 - val_loss: 1.6015
Epoch 5/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 696ms/step - accuracy: 0.8208 - loss: 1.5929 - val_accuracy: 0.7083 - val_loss: 1.5889
Epoch 6/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 688ms/step - accuracy: 0.8167 - loss: 1.5774 - val_accuracy: 0.7083 - val_loss: 1.5770
Epoch 7/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 695ms/step - accuracy: 0.8236 - loss: 1.5652 - val_accuracy: 0.7083 - val_loss: 1.5651
Epoch 8/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 31s 699ms/step - accuracy: 0.8333 - loss: 1.5563 - val_accu

In [11]:
print(history.history.keys())

dict_keys(['accuracy', 'loss'])
